In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [243]:
import http.client
import json
import pandas as pd
import csv
import psycopg2

In [247]:
import requests

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"1000"}

headers = {
	"x-rapidapi-key": "bc68487461mshca1d2214a6e1c46p1c3c47jsn5f4df3a84144",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


data= response.json()

#Saving into a file
filename = "PropertyRecords.json"
with open(filename,"w") as file:
    json.dump(data,file,indent=4)


[{'addressLine1': '843 E 30th St', 'city': 'Baltimore', 'state': 'MD', 'zipCode': '21218', 'formattedAddress': '843 E 30th St, Baltimore, MD 21218', 'county': 'Baltimore City', 'features': {}, 'id': '843-E-30th-St,-Baltimore,-MD-21218', 'longitude': -76.606186, 'latitude': 39.323861}, {'addressLine1': '822 San Juan Pl', 'city': 'San Diego', 'state': 'CA', 'zipCode': '92109', 'formattedAddress': '822 San Juan Pl, Apt 4, San Diego, CA 92109', 'addressLine2': 'Apt 4', 'bedrooms': 2, 'squareFootage': 1669, 'yearBuilt': 1984, 'county': 'San Diego', 'assessorID': '423-606-13-04', 'legalDescription': 'TR 1651 BLK 160*US 4 PER DOC85-028354&UND INT IN LOTS I&J', 'subdivision': 'MISSION BEACH', 'zoning': 'R-1:SINGLE FAM-RES', 'bathrooms': 2, 'lotSize': 4778, 'propertyType': 'Condo', 'ownerOccupied': False, 'features': {'architectureType': 'Condo / Apartment', 'garage': True, 'garageSpaces': 2, 'garageType': 'Garage', 'unitCount': 1}, 'taxAssessment': {'2022': {'value': 310439, 'land': 146663, 'i

In [249]:
#Read into a dataframe

Propertyrecord_df = pd.read_json('PropertyRecords.json')

In [251]:
Propertyrecord_df.info()
print(Propertyrecord_df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      10 non-null     object 
 1   city              10 non-null     object 
 2   state             10 non-null     object 
 3   zipCode           10 non-null     int64  
 4   formattedAddress  10 non-null     object 
 5   county            10 non-null     object 
 6   features          10 non-null     object 
 7   id                10 non-null     object 
 8   longitude         10 non-null     float64
 9   latitude          10 non-null     float64
 10  addressLine2      4 non-null      object 
 11  bedrooms          7 non-null      float64
 12  squareFootage     8 non-null      float64
 13  yearBuilt         7 non-null      float64
 14  assessorID        8 non-null      object 
 15  legalDescription  8 non-null      object 
 16  subdivision       8 non-null      object 
 17  

In [57]:
#Transformational 

#Convert columns with dictionary into string
import json

Propertyrecord_df['features']=Propertyrecord_df ['features'].apply(json.dumps)

#fill missing nd NA values
Propertyrecord_df.fillna(
    {
        'bathrooms': 0,
        'bedrooms': 0,
        'squareFootage': 0, 
        'county': "Not Available", 
        'propertyType': "unknown",
        'addressLine1': "unknown",
        'city': "unknown", 
        'state': "unknown", 
        'zipCode': "unknown", 
        'formattedAddress': "Not Available",
        'yearBuilt': 0, 
        'features': "unknown", 
        'assessorID': "unknown", 
        'legalDescription': "Not Available",
        'subdivision': "Not Available", 
        'ownerOccupied': 0, 
        'lotSize': 0, 
        'taxAssessment': "Not Available",
        'propertyTaxes': "Not Available", 
        'lastSaleDate': "Not Available", 
        'lastSalePrice': 0, 
        'owner': "unknown", 
        'id': "unknown",
        'longitude': "unknown", 
        'latitude': "unknown", 
        'zoning': "unknown", 
        'addressLine2': "Not Available"
    },
    inplace=True
)


In [59]:
print(Propertyrecord_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         10 non-null     float64
 1   bedrooms          10 non-null     float64
 2   squareFootage     10 non-null     float64
 3   county            10 non-null     object 
 4   propertyType      10 non-null     object 
 5   addressLine1      10 non-null     object 
 6   city              10 non-null     object 
 7   state             10 non-null     object 
 8   zipCode           10 non-null     int64  
 9   formattedAddress  10 non-null     object 
 10  lotSize           10 non-null     float64
 11  yearBuilt         10 non-null     float64
 12  features          10 non-null     object 
 13  assessorID        10 non-null     object 
 14  legalDescription  10 non-null     object 
 15  subdivision       10 non-null     object 
 16  zoning            10 non-null     object 
 17  

In [253]:
#create the FACT Table
fact_columns =['addressLine1','city', 'state','zipCode', 'formattedAddress',  'squareFootage', 'yearBuilt', 'bathrooms', 'bedrooms', 'lotSize', 'propertyType',
                'longitude', 'latitude']

fact_table = Propertyrecord_df[fact_columns]

#Create Location Dimension
Location_dim= Propertyrecord_df[['addressLine1','city', 'state','zipCode','county','longitude', 'latitude']].drop_duplicates().reset_index(drop=True)
Location_dim.index.name= 'location_id'

In [255]:
#Create Sales Dimension

sales_dim = Propertyrecord_df[['lastSalePrice','lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name= 'sales_id'
sales_dim.head()

,lastSalePrice,lastSaleDate
sales_id,,
0,NaN,NaN
1,NaN,2018-08-28T00:00:00.000Z
2,200000.0,2019-12-27T00:00:00.000Z
3,335000.0,2022-07-29T00:00:00.000Z
4,646900.0,2016-10-11T00:00:00.000Z


In [257]:
#Create Property Features Dimension
features_dim = Propertyrecord_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim.index.name ='features_id'
features_dim.to_csv('features_dim.csv')
features_dim.head()


TypeError: unhashable type: 'dict'

In [29]:

#Output to csv
fact_table.to_csv('property_fact.csv', index=False)
Location_dim.to_csv('Location_dimension.csv', index=True)
sales_dim.to_csv('sales_dimension.csv', index=True)
features_dim.to_csv('features_dimension.csv', index=True)

In [67]:
  #Loading Layer

# develop a function to connect to pgadmin

def get_db_connection():
    connection = psycopg2.connect(
        host= 'localhost',
        database='postgres',
        user='postgres',
        password='shakirat12'
    )
    return connection

conn = get_db_connection()


In [259]:
#creating tables


def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query='''CREATE SCHEMA IF NOT EXISTS zapbank;

                            DROP TABLE IF EXISTS zapbank.fact_table;
                            DROP TABLE IF EXISTS zapbank.location_dim;
                            DROP TABLE IF EXISTS zapbank.sales_dim;
                            DROP TABLE IF EXISTS zapbank.features_dim;

                            CREATE TABLE zapbank.fact_table(
                                addressline1 VARCHAR(255),
                                city VARCHAR(100),
                                state VARCHAR(50),
                                zipCode INTEGER,
                                formattedAddress VARCHAR(255),
                                squareFootage FLOAT,
                                yearBuilt FLOAT,
                                bathrooms FLOAT,
                                bedrooms FLOAT,
                                lotSize FLOAT,
                                propertyType VARCHAR(255),
                                longitude FLOAT,
                                latitude FLOAT
                                );

                                CREATE TABLE zapbank.location_dim(
                                    location_id SERIAL PRIMARY KEY,
                                    addressLine1 VARCHAR(255),
                                    city VARCHAR(100),
                                    state VARCHAR (50),
                                    zipCode INTEGER,
                                    county VARCHAR(100),
                                    longitude FLOAT,
                                    latitude FLOAT
                                    );

                                CREATE TABLE zapbank.sales_dim(
                                   sales_id SERIAL PRIMARY KEY,
                                   lastSalePrice FLOAT,
                                   lastSaleDate DATE
                                   );

                                CREATE TABLE zapbank.features_dim(
                                    features_id SERIAL PRIMARY KEY,
                                    features TEXT,
                                    propertyType VARCHAR(255),
                                    zoning VARCHAR(255)
                                     );'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
create_tables()

 


In [89]:
#Create a funcion to load csv data from a folder into the DB
def load_data_from_csv_to_table(csv_path,table_name):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) #Skip the header row
        for row in reader:
            placeholders = ', '.join(['%s']* len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders});"
            cursor.execute(query,row)
        conn.commit()
        cursor.close()
        conn.close()



In [91]:
# for fact table
fact_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\property_fact.csv'
load_data_from_csv_to_table(fact_csv_path, 'zapbank.fact_table')

In [93]:
# for Location dimension table
location_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\Location_dimension.csv'
load_data_from_csv_to_table(location_csv_path, 'zapbank.location_dim')

In [97]:
# for features dimension table
features_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\features_dimension.csv'
load_data_from_csv_to_table(features_csv_path, 'zapbank.features_dim')

In [264]:
 #Create a New funcion to load csv data for sales  from a folder into the DB
def load_data_from_csv_to_sales_table(csv_path,table_name):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) #Skip the header row
        for row in reader:
            #convert empty strings (or Not available) in date column to None (NULL in sql)
            row=[
              None if (cell == '' or cell =='Not Available') and col_name == 'lastSaleDate'
                 else cell for cell, col_name in zip(row, sales_dim_columns)]
        # Prepare query placeholders and execute the insert statement
        
        placeholders = ', '.join(['%s'] * len(row))
        query = f"INSERT INTO {table_name} VALUES ({placeholders});"
        cursor.execute(query, row)
        conn.commit()
        cursor.close()
        conn.close()
   

#define the columns names in sales_dim table
sales_dim_columns = ['sales_id','lastSalePrice', 'lastSaleDate']



In [324]:
import pandas as pd
from datetime import datetime
import psycopg2
from psycopg2 import Error

def load_data_from_csv_to_sales_table(csv_path, table_name):
    """
    Load sales data from CSV to database with proper data type handling and error checking.
    """
    try:
        # Get database connection
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Read the CSV file using pandas
        df = pd.read_csv(csv_path)
        print("Original CSV Data:")
        print(df)
        
        # Data type conversions
        if 'lastSalePrice' in df.columns:
            # Convert to float, replacing any non-numeric values with None
            df['lastSalePrice'] = pd.to_numeric(df['lastSalePrice'], errors='coerce')
        
        if 'lastSaleDate' in df.columns:
            # Replace empty strings or 'Not Available' with None
            df['lastSaleDate'] = df['lastSaleDate'].replace(['', 'Not Available'], None)
            # Convert to datetime, invalid parsing becomes None
            df['lastSaleDate'] = pd.to_datetime(df['lastSaleDate'], errors='coerce')
            # Convert timezone-aware timestamps to timezone-naive
            df['lastSaleDate'] = df['lastSaleDate'].dt.tz_localize(None)

        print(df)
        # Iterate over each row and insert into the database
        successful_inserts = 0
        failed_inserts = 0
        db_columns = list(df.columns)
        for index, row in df.iterrows():
            try:
                print(row)
                # Convert row to list and handle None values
                row_data = [None if pd.isna(val) else val for val in row]
                
                # Prepare the insert statement
                placeholders = ', '.join(['%s'] * len(row_data))
                query = f"INSERT INTO {table_name} ({', '.join(db_columns)}) VALUES ({placeholders});"
                
                print(f"\nAttempting insert for row {index + 1}:")
                print(f"Query: {query}")
                print(f"Data: {row_data}")
                
                cursor.execute(query, row_data)
                successful_inserts += 1
                
            except Error as e:
                failed_inserts += 1
                print(f"\nError inserting row {index + 1}:")
                print(f"Data: {row_data}")
                print(f"Error: {str(e)}")
                continue
        
        # Commit the transaction
        conn.commit()
        
        print(f"\nInsert Summary:")
        print(f"Successful inserts: {successful_inserts}")
        print(f"Failed inserts: {failed_inserts}")
        
    except Error as e:
        print(f"Database error: {str(e)}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"General error: {str(e)}")
        if conn:
            conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()




In [328]:
# Then load the data
load_data_from_csv_to_sales_table(sales_csv_path, 'zapbank.sales_dim')

Original CSV Data:
   lastSalePrice              lastSaleDate
0       783991.0  2013-04-29T00:00:00.000Z
1            0.0  2021-07-13T00:00:00.000Z
2            0.0             Not Available
3       629000.0  2016-10-12T00:00:00.000Z
4       536005.0  2018-09-28T00:00:00.000Z
   lastSalePrice lastSaleDate
0       783991.0   2013-04-29
1            0.0   2021-07-13
2            0.0          NaT
3       629000.0   2016-10-12
4       536005.0   2018-09-28
lastSalePrice               783991.0
lastSaleDate     2013-04-29 00:00:00
Name: 0, dtype: object

Attempting insert for row 1:
Query: INSERT INTO zapbank.sales_dim (lastSalePrice, lastSaleDate) VALUES (%s, %s);
Data: [783991.0, Timestamp('2013-04-29 00:00:00')]
lastSalePrice                    0.0
lastSaleDate     2021-07-13 00:00:00
Name: 1, dtype: object

Attempting insert for row 2:
Query: INSERT INTO zapbank.sales_dim (lastSalePrice, lastSaleDate) VALUES (%s, %s);
Data: [0.0, Timestamp('2021-07-13 00:00:00')]
lastSalePrice    0.0
las

In [330]:
# for sales dimension table
sales_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\sales_dimension.csv'
load_data_from_csv_to_sales_table(sales_csv_path, 'zapbank.sales_dim')

Original CSV Data:
   lastSalePrice              lastSaleDate
0       783991.0  2013-04-29T00:00:00.000Z
1            0.0  2021-07-13T00:00:00.000Z
2            0.0             Not Available
3       629000.0  2016-10-12T00:00:00.000Z
4       536005.0  2018-09-28T00:00:00.000Z
   lastSalePrice lastSaleDate
0       783991.0   2013-04-29
1            0.0   2021-07-13
2            0.0          NaT
3       629000.0   2016-10-12
4       536005.0   2018-09-28
lastSalePrice               783991.0
lastSaleDate     2013-04-29 00:00:00
Name: 0, dtype: object

Attempting insert for row 1:
Query: INSERT INTO zapbank.sales_dim (lastSalePrice, lastSaleDate) VALUES (%s, %s);
Data: [783991.0, Timestamp('2013-04-29 00:00:00')]
lastSalePrice                    0.0
lastSaleDate     2021-07-13 00:00:00
Name: 1, dtype: object

Attempting insert for row 2:
Query: INSERT INTO zapbank.sales_dim (lastSalePrice, lastSaleDate) VALUES (%s, %s);
Data: [0.0, Timestamp('2021-07-13 00:00:00')]
lastSalePrice    0.0
las

In [270]:
def load_data_from_csv_to_sales_table(csv_path, table_name):
    import csv

    conn = get_db_connection()
    cursor = conn.cursor()

    # Define column names
    sales_dim_columns = ['sales_id', 'lastSalePrice', 'lastSaleDate']

    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row

        for row in reader:
            # Convert values to match table schema
            row = [
                int(cell) if col_name == 'sales_id' and cell.isdigit() else  # Convert sales_id to integer
                float(cell) if col_name == 'lastSalePrice' and cell.replace('.', '', 1).isdigit() else  # Convert lastSalePrice to float
                None if (cell == '' or cell == 'Not Available') and col_name == 'lastSaleDate' else  # Convert empty or unavailable dates to None
                cell for cell, col_name in zip(row, sales_dim_columns)
            ]

            # Prepare query placeholders and execute the insert statement
            placeholders = ', '.join(['%s'] * len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders});"
            cursor.execute(query, row)

    conn.commit()
    cursor.close()
    conn.close()


In [127]:
print('All data has been loaded successfully into their respective schema and table')

All data has been loaded successfully into their respective schema and table
